In [1]:
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os, sys
from itertools import product
USER = os.getenv("USER")
sys.path.append(f'/home/{USER}/repos/agriculture/1_code/5_crop_shift')
from agval.utils.ag_weights import snag_weights
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_context('poster')

/home/kschwarz/miniconda3/envs/dscim2/lib/python3.8/site-packages/statsmodels/compat/pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
USA=True
models=['low']
rcps=['rcp85', 'rcp45']
ssps=['SSP3', 'SSP2']
years=[2099]
slrs=[5]
sectors=['CAMEL', 'AMEL', 'agriculture', 'energy', 'labor', 'coastal', 'mortality'] 
output = '/mnt/CIL_integration/plots/damages_income_share_boxplots'

In [3]:
model_dict = {
    'low' : 'IIASA GDP',
    'high' : 'OECD Env-Growth'
}

In [4]:
for sector, model, rcp, ssp, year, slr in product(
    sectors,
    models,
    rcps,
    ssps,
    years, 
    slrs,
):
    
    selection = dict(ssp=ssp, model=model_dict[model], year=year, rcp=rcp, slr=slr)
    
    if sector=='CAMEL':
        ce_cc = xr.open_zarr(
            f"/shares/gcp/integration/CE_library_epa_vsl_bc39/synthetic_CAMEL/CAMEL_weighted_gcm_slr{slr}_{year}_adding_up_ce_cc_eta2.0.zarr"
        )
    else:
        ce_cc = xr.open_zarr(
            f"/shares/gcp/integration/CE_library_epa_vsl_bc39/{sector}/adding_up_ce_cc_eta2.0.zarr"
        )
    ce_cc = ce_cc.sel({k:v for k,v in selection.items() if k in ce_cc.dims})

    gdppc = xr.open_zarr(
    "/shares/gcp/integration/float32/dscim_input_data/econvars/zarrs/integration-econ-bc39.zarr"
)

    gdppc = gdppc.chunk(
        {'region' : len(gdppc.region.values)}
    ).sel(
        {k:v for k,v in selection.items() if k in gdppc.dims}
    ).gdppc
    
    # undo the mean consumption calculation (which is gdppc - damages) to get back damages
    mean_damages = -1*(ce_cc.ce_cc - gdppc)
    
    if USA == True:
        mean_damages = mean_damages.sel(region=[i for i in ce_cc.region.values if 'USA.' in i])
        gdppc = gdppc.sel(region=[i for i in gdppc.region.values if 'USA.' in i])
    
    if sector=='coastal':
        rcp=''
    elif sector=='CAMEL':
        pass
    else:
        slr=''
        weights = snag_weights(rcp).to_xarray().weight
        mean_damages['gcm'] = mean_damages.gcm.str.upper()
        mean_damages = mean_damages.weighted(weights).mean('gcm')
    
    damages_inc_share = (mean_damages/gdppc).chunk({'region' : len(gdppc.region.values)}).to_dataset(name='damages_inc_share')
    damages_inc_share['gdppc'] = gdppc
    bins = gdppc.quantile(np.linspace(0,1,6)).values
    
    # table
    groupby = damages_inc_share.groupby_bins('gdppc', bins)
    mean = groupby.mean('region').damages_inc_share
    quants = groupby.quantile([0, 0.25, 0.75, 1], 'region').damages_inc_share
    table = xr.concat([quants, mean.expand_dims({'quantile' : ['mean']})], 'quantile').to_dataframe()
    table.to_csv(f"{output}/{sector}_{model_dict[model]}_{ssp}_{rcp}{slr}_{year}_domestic-{USA}.csv")
    
    # plot
    damages_inc_share['quintile'] = np.nan

    for i, b in enumerate(bins):

        if i > 0:
            damages_inc_share['quintile'] = xr.where(
                (damages_inc_share.gdppc < bins[i]) & (damages_inc_share.gdppc >= bins[i-1]),
                i,
                damages_inc_share.quintile
            )
        
    plot = damages_inc_share.to_dataframe().reset_index()
    plot = plot.loc[~plot.quintile.isnull()]
    plot['quintile'] = plot.quintile.astype(int)

    fig, ax = plt.subplots(figsize=(10, 10))
    fig.suptitle(f"Damages due to climate change as a share of GDP, \n by GDP quintiles across regions")

    sns.boxplot(data=plot,
                y='damages_inc_share',
                x='quintile',
                hue='quintile',
                dodge=False,
                whis=(5, 95),
                ax=ax,
                width=0.75,
                palette='husl',
                fliersize=2,
                saturation=1,
               )

    plt.xlabel("Quintile")
    plt.ylabel("Damages as a share of GDP")
    ax.get_legend().remove()
    plt.savefig(f"{output}/{sector}_{model_dict[model]}_{ssp}_{rcp}{slr}_{year}_domestic-{USA}.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    print(sector, rcp, ssp, model)

CAMEL rcp85 SSP3 low
CAMEL rcp85 SSP2 low
CAMEL rcp45 SSP3 low
CAMEL rcp45 SSP2 low
AMEL rcp85 SSP3 low
AMEL rcp85 SSP2 low
AMEL rcp45 SSP3 low
AMEL rcp45 SSP2 low
agriculture rcp85 SSP3 low
agriculture rcp85 SSP2 low
agriculture rcp45 SSP3 low
agriculture rcp45 SSP2 low
energy rcp85 SSP3 low
energy rcp85 SSP2 low
energy rcp45 SSP3 low
energy rcp45 SSP2 low
labor rcp85 SSP3 low
labor rcp85 SSP2 low
labor rcp45 SSP3 low
labor rcp45 SSP2 low
coastal  SSP3 low
coastal  SSP2 low
coastal  SSP3 low
coastal  SSP2 low
mortality rcp85 SSP3 low
mortality rcp85 SSP2 low
mortality rcp45 SSP3 low
mortality rcp45 SSP2 low
